In [2]:
from flask import Flask, request, url_for, send_file, render_template, jsonify
from werkzeug.utils import secure_filename
import os
import threading
import nest_asyncio
import pandas as pd
import pdfplumber
import tabula
import re
from collections import defaultdict
import fitz
import logging

nest_asyncio.apply()

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['PROCESSED_FOLDER'] = 'processed'
app.config['ALLOWED_EXTENSIONS'] = {'pdf'}
app.secret_key = 'supersecretkey'

for folder in [app.config['UPLOAD_FOLDER'], app.config['PROCESSED_FOLDER']]:
    if not os.path.exists(folder):
        os.makedirs(folder)

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']

@app.route('/')
def upload_form():
    return render_template('upload.html')

@app.route('/upload', methods=['POST'])
def upload_folder():
    folder_type = request.form.get('folder_type')
    customer_name = request.form.get('customer_name', '')  # Get customer_name if provided (for OBI)
    password = request.form.get('password', '')  # Get the password from the form

    files = request.files.getlist('files[]')

    if not folder_type or not files:
        return jsonify({'message': 'Please select a folder type and upload files.'}), 400

    response = {'message': 'Files processed successfully', 'error_files': []}

    if folder_type == 'adeo':
        processed_files, error_files = process_files(files, process_adeo_file)
    elif folder_type == 'obi':
        processed_files, error_files = process_files(files, process_obi_file, customer_name=customer_name)
    elif folder_type=='CAMS':
        processed_files, error_files = process_files(files, CAMS_file, password=password)
    elif folder_type=='KFINTECH':
        processed_files, error_files = process_files(files, kfintech_file, password=password)
    else:
        return jsonify({'message': 'Invalid folder type selected.'}), 400

    if processed_files:
        combined_excel = generate_combined_excel(processed_files, f'{folder_type}_combined_output.xlsx')
        response['download_url'] = url_for('download_file', filename=combined_excel)


    response['error_files'] = error_files

    return jsonify(response)

def process_files(files, process_function, **kwargs):
    processed_files = []
    error_files = []
    for file in files:
        if allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(file_path)
            try:
                output_filename = process_function(file_path, **kwargs)
                processed_files.append(output_filename)
            except Exception as e:
                app.logger.error(f"Error processing file {filename}: {e}")
                error_files.append(filename)
        else:
            error_files.append(file.filename)
    return processed_files, error_files
def process_adeo_file(file_path):
    # Your existing process_file code renamed to process_adeo_file
    try:
        text = ""
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"

        regex_patterns = (
            (r'(CO\d*\d)', 'PO No'),
            (r'(Type : (\w*\w))', 'Type'),
            (r'(Supplier : (\d+))', 'Supplier'),
            (r'(Validation date : (\d{2}/\d{2}/\d{4}))', 'Validation date'),
            (r'(Purchase Incoterm Place : (\d{2}/\d{2}/\d{4}))', 'Purchase Incoterm Place'),
            (r'(Ordered for : (\d+))', 'Ordered for'),
            (r'(Amount : (?:(\d+)(\s+))?(\d+(\.\d+)?)\s+([a-zA-Z]+))', 'Amount'),
            (r'(Amount : (?:\d+\s+)?(\d+(\.\d+)?)\s+([a-zA-Z]+))', 'Currency'),
            (r'(Delivery to : (\d+))', 'Delivery to'),
            (r'(Blanket Order No : (\w+))', 'Blanket Order No')
        )
        compiled_patterns = [(re.compile(pattern), label) for pattern, label in regex_patterns]
        habi = ''
        matches = []
        for pattern, label in compiled_patterns:
            match = pattern.search(text)
            if match:
                if label == 'PO No':
                    matches.append(match.group(1))
                    habi = match.group(1)
                elif label == 'Currency':
                    matches.append(match.group(4))
                elif label == 'Amount':
                    if match.group(3):
                        thousands = match.group(2)
                        hundreds = match.group(4)
                        total = thousands + hundreds
                        matches.append(total)
                    else:
                        hundreds = match.group(4)
                        matches.append(hundreds)
                else:
                    matches.append(match.group(2))
        if not matches:
            raise ValueError("No matches found for required fields.")
        matches_tuple = tuple(matches)
        d = defaultdict(lambda: '')
        list = ['PO No', 'Type', 'Supplier', 'Validation date', 'Purchase Incoterm Place', 'Ordered for', 'Amount', 'Currency', 'Delivery to', 'Blanket Order No']
        for i in range(len(matches_tuple)):
            d[list[i]] = matches_tuple[i]
        output_csv_path = os.path.join(app.config['UPLOAD_FOLDER'], 'tables.csv')
        tabula.convert_into(file_path, output_csv_path, output_format="csv", pages='all', stream=True)
        df_tables = pd.read_csv(output_csv_path, encoding='latin-1')
        if len(df_tables) > 33:
            df_subset = df_tables.iloc[33:]
            df_subset.drop(df_subset.columns[df_subset.columns.str.contains('unnamed', case=False)], axis=1, inplace=True)
            two_row = df_subset.drop(columns=['Supplier Ref.', 'Description', 'GTIN', 'Dim. l*w*h cm', 'Weight Kg', 'Quantity','Amount'])
            two_row.rename(columns={
                'PCB': 'Quantity',
                'Price' : 'Total',
                'Nb Master': 'Price',
            }, inplace=True)
            df_sub = df_tables.iloc[:33]
            df_sub.drop(df_sub.columns[df_sub.columns.str.contains('unnamed', case=False)], axis=1, inplace=True)
            one_row = df_sub.drop(columns=['Supplier Ref.', 'Description', 'GTIN', 'Dim. l*w*h cm', 'Weight Kg', 'PCB', 'Nb Master'])
            one_row.rename(columns = {
                'Amount':'Total',
            },inplace = True)
            data = pd.concat([one_row, two_row], ignore_index=False, sort=False, axis=0)
        else:
            df_tables.drop(df_tables.columns[df_tables.columns.str.contains('unnamed', case=False)], axis=1, inplace=True)
            df_tables.rename(columns = {
                'Amount':'Total',
            },inplace = True)
            data = df_tables.drop(columns=['Supplier Ref.', 'Description', 'GTIN', 'Dim. l*w*h cm', 'Weight Kg', 'PCB', 'Nb Master'])
        df_tables = data
        df_tables.dropna(inplace=True)
        df_tables['Customer Ref.'] = df_tables['Customer Ref.'].astype(int)
        df_tables.reset_index(drop=True, inplace=True)
        details = [d.copy() for _ in range(len(df_tables))]
        df_2 = pd.DataFrame(details)
        df_final = pd.concat([df_2, df_tables], ignore_index=False, sort=False, axis=1)
        df_final.rename(columns={
            'ADEO Key': 'Item_No',
            'Ordered for': 'BU',
            'Delivery to': 'BU_Delivery',
            'Validation date': 'Buyer_PO_Date',
            'Purchase Incoterm Place': 'Ship_Date',
            'Supplier': 'Supplier_Code',
            'Price': 'Unit_Price',
            'Quantity': 'Qty'
        }, inplace=True)
        first_half = df_final[['PO No', 'Supplier_Code', 'BU', 'BU_Delivery', 'Buyer_PO_Date', 'Ship_Date', 'Item_No', 'Unit_Price', 'Qty','Total']]
        first_half['Qty'] = first_half['Qty'].astype(str)
        first_half['Qty'] = first_half['Qty'].str.replace(r'\s+(\d)', r'\1', regex=True)
        first_half['Unit_Price'] = first_half['Unit_Price'].astype(str)
        first_half['Unit_Price'] = first_half['Unit_Price'].str.replace(r'\s+(\d)', r'\1', regex=True)
        first_half['Total'] = first_half['Total'].astype(str)
        first_half['Total'] = first_half['Total'].str.replace(r'\s+(\d)', r'\1', regex=True)
        second_half = df_final[['Type', 'Amount', 'Currency', 'Blanket Order No', 'Customer Ref.']]
        format = pd.concat([first_half, second_half], ignore_index=False, sort=False, axis=1)
        final_output_path = os.path.join(app.config['UPLOAD_FOLDER'], f'{habi}.csv')
        format.to_csv(final_output_path, index=False)
        return f'{habi}.csv'
    except Exception as e:
        app.logger.error(f"Error processing file: {e}")
        raise e

def process_obi_file(file_path, customer_name=''):
    try:
        # Extract text from the PDF using pdfplumber
        with pdfplumber.open(file_path) as pdf:
            all_text = ""
            for page in pdf.pages:
                text = page.extract_text()
                all_text += text + "\n"  # add a newline to separate pages (adjust as needed)
        lst = all_text.split('\n')
        hbi=''
        pattern_5=re.compile(r'(Passwort:\w+)')
        for line in all_text.split('\n'):
            if pattern_5.match(line):
                hbi=line
        if lst[0] != 'O R D E R' and hbi:
            # Processing for OBI-Lux Tools
            # Extracting order number
            matches = lst[2].split()[-1]
            # Extracting vendor
            vendor = lst[3]
            s = vendor.split()
            if 'Order' in s:
                vendor = lst[2]
                eg = vendor.split(' Order')
                vendor = eg[0]
            # Compiling data into a dictionary
            from collections import defaultdict
            data_dict = defaultdict(str)
            data_dict['Po No'] = matches
            data_dict['Vendor_Code'] = vendor

            # Extracting table data with regex
            original_pattern = re.compile(
                r'\d{1}?\s?'       # match a sequence of digits followed by spaces
                r'\d+\s+'           # match another sequence of digits followed by spaces
                r'\w+(\s+\w+)*\s*'  # match words with optional additional words separated by spaces, followed by optional spaces
            )

            combined_pattern = re.compile(
                fr'{original_pattern.pattern}'  # include the original pattern as mandatory
                r'(?:(?:\d{4}-\d{2}-\d{2}|\b\w+\b|\s)+)?'  # optionally match dates, words, or spaces
                r'\b\d{1,4}\.\d{2}\b'  # match a decimal number with 1-4 digits before the decimal point and 2 digits after
            )

            res = []
            for line in all_text.split('\n'):
                if combined_pattern.findall(line):
                    res.append(line)
            df = []
            for i in range(len(res)):
                val = res[i].split()
                tmp = []
                tmp.append(val[1])
                for i in range(len(val)-6, len(val)):
                    tmp.append(val[i])
                df.append(tmp)

            # Creating DataFrame
            data = pd.DataFrame(df, columns=['Item_No', 'Ship Date', 'Quantity', 'Unit Price', 'Package', 'Price unit Qty', 'Total'])
            data = data.drop(columns=['Package'])
            data['Unit Price'] = data['Unit Price'].astype(str).str.replace(',', '.')
            data['Total'] = data['Total'].astype(str).str.replace('.', '')
            data['Total'] = data['Total'].astype(str).str.replace(',', '.')
            if customer_name:
                data['Customer'] = 'CUST0028'
            # Adding additional details
            details = [data_dict.copy() for _ in range(len(data))]
            details_df = pd.DataFrame(details)
            df_final = pd.concat([data, details_df], ignore_index=False, sort=False, axis=1)
            # Saving final CSV
            final_output_path = os.path.join(app.config['UPLOAD_FOLDER'], f'{matches}.csv')
            df_final.to_csv(final_output_path, index=False)
            return f'{matches}.csv'
        elif lst[0] == 'O R D E R':
            regex_patterns = (
            (r'(Supplier-No.: (\d+))', 'Supplier-No'),
            (r'(Order-No.: (\d+))', 'Order-No'),
            (r'(Dated: (\d{2}.\d{2}.\d{2}))', 'Date')
            )
            compiled_patterns = [(re.compile(pattern), label) for pattern, label in regex_patterns]
            mat = []
            for pattern, label in compiled_patterns:
                matc= pattern.search(all_text)
                if matc:
                    mat.append(matc.group(2))
            matches_tuple = tuple(mat)
            if not matches_tuple:
                raise ValueError("No matches found for required fields.")
            from collections import defaultdict
            d = defaultdict(lambda: '')
            list = ['Supplier-No','Order-No','Date']
            for i in range(len(matches_tuple)):
                d[list[i]] = matches_tuple[i]
            pattern_1=re.compile(r'(\d{3,}\s(.*)[A-Z]{3}$)')
            pattern_3=re.compile(r'^EUROMATE\b.*')
            matches=[]
            ans=''
            for line in all_text.split('\n'):
                if pattern_1.match(line):
                    matches.append(line)
                if pattern_3.match(line):
                    ans=line
            li=[]
            for i in range(len(matches)):
                tex=matches[i].split()
                li.append(tex[-2])
            pattern_2 = r'\d+(\.\d+)?(?=/)'
            for i in range(len(li)):
                te=li[i]
                match = re.search(pattern_2, te)
                if match:
                    number = match.group()
                    li[i]=number
            df=[]
            for i in range(len(matches)):
                val=matches[i].split()
                tmp=[]
                tmp.append(val[0])
                tmp.append(val[-4])
                tmp.append(li[i])
                df.append(tmp)
            data = pd.DataFrame(df,columns=['Item_No','Quantity','Price'])
            if customer_name:
                if ans:
                    data['Customer'] = 'CUST0027'
                else:
                    data['Customer']='CUST0028'
            details = [d.copy() for _ in range(len(data))]
            a = pd.DataFrame(details)
            df_merged = pd.concat([data,a], ignore_index=False, sort=False,axis=1)
            df_merged['Date'] = df_merged['Date'].astype(str).str.replace('.', '-')
            import numpy as np
            df_merged['Ship-No']=np.nan
            final_output_path = os.path.join(app.config['UPLOAD_FOLDER'], f'{matches_tuple[0]}_euromate.csv')
            df_merged.to_csv(final_output_path, index=False)
            return f'{matches_tuple[0]}_euromate.csv'
        else:
            raise ValueError(f"Unsupported customer name: {customer_name}")
    except Exception as e:
        app.logger.error(f"Error processing OBI file: {e}")
        raise e
def CAMS_file(file_path, password=None):
    try:
        text = ""
        with pdfplumber.open(file_path, password=password) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"
        start_pattern = re.compile(r'^Folio No:')
        end_pattern = re.compile(r'^Closing Unit Balance:')
        date_pattern = re.compile(r'\b\d{2}-[A-Za-z]{3}-\d{4}\b')  # Pattern for DD-MMM-YYYY format
        date_pattern_2 = re.compile(r'\b\d{2}-[A-Za-z]{3}-\d{4}\b\s+(.*)\s+\b\d{2}-[A-Za-z]{3}-\d{4}\b')
        folio_pattern = re.compile(r'Folio No:(.*?)(?:PAN:\s*([A-Za-z]{5}\d{4}[A-Za-z])|KYC:)')
        pan_pattern = re.compile(r'PAN:\s*([A-Za-z]{5}\d{4}[A-Za-z])')
        within_lines = False
        extracted_lines = []
        for line in text.split('\n'):
            if start_pattern.match(line):
                if extracted_lines and extracted_lines[-1] != "":
                    extracted_lines.append("")  # Add an empty line as delimiter
                within_lines = True
            if within_lines and not end_pattern.match(line):
                extracted_lines.append(line.strip())
            if end_pattern.match(line):
                within_lines = False
        sections = []
        current_section = []
        folio_numbers = []
        pan_numbers = []
        
        for line in extracted_lines:
            if line == "":
                if current_section:  # Only append if there are lines in the current section
                    sections.append(current_section)
                    current_section = []  # Reset current_section for the next section
            else:
                current_section.append(line)
        
        # Append the last section if it exists
        if current_section:
            sections.append(current_section)
        filtered_sections = []
        for section in sections:
            filtered_section = []
            folio_number = None
            pan_number = None
            for line in section:
                if folio_pattern.search(line):
                    folio_number = folio_pattern.search(line).group(1).strip()
                if pan_pattern.search(line):
                    pan_number = pan_pattern.search(line).group(1)
                if date_pattern.match(line):
                    if date_pattern_2.match(line):  # Check again to skip "date to date" expressions
                        continue
                    else:
                        filtered_section.append(line)
            if filtered_section:
                filtered_sections.append((folio_number, pan_number, filtered_section))
        df_rows = []

    # Extract data and format it for DataFrame
        for folio_number, pan_number, section in filtered_sections:
            for line in section:
                parts = line.split()
                date = parts[0]
                transaction_parts = []
                numeric_parts = []
                for part in parts[1:]:
                    if re.match(r'^\([-+]?[0-9]*\.?[0-9]+(?:,\d{3})*(?:\.\d+)?\)$', part):  # Check for numbers in parentheses
                        numeric_parts.append(part.strip('()'))  # Remove parentheses
                        if part.startswith('(') and part.endswith(')'):  # Check if it's a negative number in parentheses
                            numeric_parts[-1] = '-' + numeric_parts[-1]  # Add negative sign
                    elif re.match(r'^[-+]?[0-9]*\.?[0-9]+(?:,\d{3})*(?:\.\d+)?$', part):  # Check for regular numbers
                        numeric_parts.append(part)
                    else:
                        transaction_parts.append(part)  # Add non-numeric parts to transaction parts
                
                transaction = " ".join(transaction_parts)
                
                amount = numeric_parts[0] if len(numeric_parts) > 0 else ''
                units = numeric_parts[1] if len(numeric_parts) > 1 else ''
                nav = numeric_parts[2] if len(numeric_parts) > 2 else ''
                unit_balance = numeric_parts[3] if len(numeric_parts) > 3 else ''
                
                df_rows.append([folio_number, pan_number, date, transaction, amount, units, nav, unit_balance])
    
        # Create a DataFrame
        df = pd.DataFrame(df_rows, columns=['Folio Number', 'PAN Number', 'Date', 'Transaction', 'Amount', 'Units', 'NAV', 'Unit_Balance'])
        final_output_path = os.path.join(app.config['UPLOAD_FOLDER'], 'habi.csv')
        df.to_csv(final_output_path, index=False)
        return 'habi.csv'
    except Exception as e:
        app.logger.error(f"Error processing file: {e}")
        raise e

def decrypt_and_extract_text(input_pdf_path, password=None):
    try:
        # Open encrypted PDF file with PyMuPDF
        pdf_document = fitz.open(input_pdf_path)

        # Check if the PDF is encrypted
        if pdf_document.is_encrypted:
            # Authenticate and decrypt the PDF
            if password and pdf_document.authenticate(password):
                # Save the decrypted PDF
                decrypted_pdf_path = input_pdf_path.replace('.pdf', '_decrypted.pdf')
                pdf_document.save(decrypted_pdf_path)
                logging.info("File decrypted successfully.")

                # Extract text from decrypted PDF
                extracted_text = extract_text_from_pdf(decrypted_pdf_path)
                return extracted_text
            elif not password:
                logging.error("Password is required to decrypt the file.")
                raise ValueError("Password is required to decrypt the file.")
            else:
                logging.error("Incorrect password or unable to decrypt.")
                raise ValueError("Incorrect password or unable to decrypt.")
        else:
            logging.info("File is not encrypted.")

            # Extract text directly from the PDF
            extracted_text = extract_text_from_pdf(input_pdf_path)
            return extracted_text

    except Exception as e:
        logging.error(f"Error processing file: {e}")
        raise e
    finally:
        if 'pdf_document' in locals():
            pdf_document.close()
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf_document:
        for page_num in range(len(pdf_document)):
            page = pdf_document.load_page(page_num)
            text += page.get_text()
    return text
def kfintech_file(file_path, password=None):
    try:
        extracted_text = decrypt_and_extract_text(file_path, password=password)
        if extracted_text:
            pattern = re.compile(
                r'PAN: (.*?)(Closing Unit Balance:)',  # Capture PAN number and content between "PAN:" and "Closing balance:"
                re.DOTALL  # Make the dot match newlines
            )

            matches = pattern.findall(extracted_text)
            extracted_content = []
            for match in matches:
                folio_number, content = match
                extracted_content.append(f"PAN: {folio_number.strip()}\nContent:\n{content.strip()}\n")

            extracted_text_without_function = "\n".join(extracted_content).strip()
            text_with_spaces = extracted_text_without_function.replace('\n', ' ')
            processed_text=text_with_spaces.replace(' PAN:', '\nPAN:')
            clean_text = re.sub(r'\s+NAV.*$', '', processed_text, flags=re.MULTILINE)
            date_pattern = r'(\d{2}-\w{3}-\d{4})'
            parts = re.split(date_pattern, clean_text)
        
            # Initialize variables
            cleaned_text = parts[0].strip()
            previous_date = None
            
            # Process the text
            for i in range(1, len(parts), 2):
                current_date = parts[i]
                details = parts[i + 1].strip()
                # Check if details start with 'PAN:' or the line starts with a date
                if (current_date and not cleaned_text.strip().endswith(current_date)):
                    cleaned_text += '\n' + current_date + ' ' + details
                else:
                    cleaned_text += ' ' + details
                
                previous_date = current_date
            
            # Print the cleaned text
            clean=cleaned_text.strip()
            pattern = r'\b\d{2}-[A-Za-z]{3}-\d{4}\s+(PAN: [A-Z]{5}[0-9]{4}[A-Z])'
            result = re.sub(pattern, r'\1', clean)
            final=result.strip()  # Strip to remove extra whitespace around the text
            start_pattern = re.compile(r'^PAN:')
            end_pattern = re.compile(r'^Closing Unit Balance:')
            date_pattern = re.compile(r'\b\d{2}-[A-Za-z]{3}-\d{4}\b')  # Pattern for DD-MMM-YYYY format
            date_range_pattern = re.compile(r'\b\d{2}-[A-Za-z]{3}-\d{4}\b\s+(.*)\s+\b\d{2}-[A-Za-z]{3}-\d{4}\b')
            folio_pattern = re.compile(r'Folio No :(.*)KYC :')
            pan_pattern = re.compile(r'PAN:\s*([A-Za-z]{5}\d{4}[A-Za-z])')
            
            # Flag to track whether we are within the desired lines
            within_lines = False
            extracted_lines = []
            for line in final.split('\n'):
                if start_pattern.match(line):
                    if extracted_lines and extracted_lines[-1] != "":
                        extracted_lines.append("")  # Add an empty line as delimiter
                    within_lines = True
                if within_lines and not date_range_pattern.search(line):  # Exclude lines matching date range pattern
                    extracted_lines.append(line.strip())
                if end_pattern.match(line):
                    within_lines = False
            
            # Create a new list of sections based on the delimiter
            sections = []
            current_section = []
            
            for line in extracted_lines:
                if line == "":
                    if current_section:  # Only append if there are lines in the current section
                        sections.append(current_section)
                        current_section = []  # Reset current_section for the next section
                else:
                    current_section.append(line)
            
            # Append the last section if it exists
            if current_section:
                sections.append(current_section)
            filtered_sections = []
            for section in sections:
                filtered_section = []
                folio_number = None
                pan_number = None
                skip_section = False
                for line in section:
                    if date_range_pattern.search(line):
                        skip_section = True
                        break
                    if folio_pattern.search(line):
                        folio_number = folio_pattern.search(line).group(1)
                    if pan_pattern.search(line):
                        pan_number = pan_pattern.search(line).group(1)
                    if date_pattern.match(line):
                        if date_range_pattern.match(line):  # Check again to skip "date to date" expressions
                            continue
                        else:
                            filtered_section.append(line)
                if filtered_section and not skip_section:
                    filtered_sections.append((folio_number, pan_number, filtered_section))
            date_pattern_n = re.compile(r"^\d{2}-[A-Za-z]{3}-\d{4}$")
            for i in range(len(filtered_sections)):
                filtered_sections[i] = (
                    filtered_sections[i][0],
                    filtered_sections[i][1],
                    [entry for entry in filtered_sections[i][2] if not date_pattern_n.match(entry)]
                )
            df_rows = []
            for folio_number, pan_number, section in filtered_sections:
                i = 0
                while i < len(section):
                    line = section[i]
                    parts = line.split()
                    date = parts[0]
                    transaction_parts = []
                    numeric_parts = []
                    
                    for part in parts[1:]:
                        if re.match(r'^\([-+]?(?:\d{1,3}(?:,\d{2,3})*|\d+)(?:\.\d+)?\)$|^\([-+]?(?:\d{1,3}(?:,\d{2,3})*|\d+)\.\d+\)$', part):  # Check for numbers in parentheses
                            numeric_parts.append(part.strip('()'))  # Remove parentheses
                            if part.startswith('(') and part.endswith(')'):  # Check if it's a negative number in parentheses
                                numeric_parts[-1] = '-' + numeric_parts[-1]  # Add negative sign
                        elif re.match(r'^[-+]?(?:\d{1,3}(?:,\d{2,3})*|\d+)(?:\.\d+)?$', part):  # Check for regular numbers
                            numeric_parts.append(part)
                        else:
                            transaction_parts.append(part)  # Add non-numeric parts to transaction parts
                    
                    transaction = " ".join(transaction_parts)
                    
                    # Check if transaction contains only "To"
                    if transaction.strip().lower() == "to":
                        i += 2  # Skip this line and the next line
                        continue
                    amount = numeric_parts[0] if len(numeric_parts) > 0 else ''
                    units = numeric_parts[1] if len(numeric_parts) > 1 else ''
                    nav = numeric_parts[2] if len(numeric_parts) > 2 else ''
                    unit_balance = numeric_parts[3] if len(numeric_parts) > 3 else ''
                    
                    # Append row to df_rows
                    df_rows.append([folio_number, pan_number, date, transaction, amount, units, nav, unit_balance])
                    
                    i += 1
    
            # Create DataFrame df
            df = pd.DataFrame(df_rows, columns=['Folio Number', 'PAN Number', 'Date', 'Transaction', 'Amount', 'Units', 'NAV', 'Unit_Balance'])
            final_output_path = os.path.join(app.config['UPLOAD_FOLDER'], 'habi.csv')
            df.to_csv(final_output_path, index=False)
            return 'habi.csv'
    except Exception as e:
        app.logger.error(f"Error processing file: {e}")
        raise e

def generate_combined_excel(csv_files, output_filename):
    combined_df = pd.concat([pd.read_csv(os.path.join(app.config['UPLOAD_FOLDER'], csv_file)) for csv_file in csv_files], ignore_index=True)
    combined_excel_path = os.path.join(app.config['PROCESSED_FOLDER'], output_filename)
    combined_df.to_excel(combined_excel_path, index=False)
    return output_filename
@app.route('/download/<filename>')
def download_file(filename):
    return send_file(os.path.join(app.config['PROCESSED_FOLDER'], filename), as_attachment=True)
    

def run_app():
    app.run(host='0.0.0.0', port=7043)

if __name__ == '__main__':
    threading.Thread(target=run_app).start()



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:7043
 * Running on http://10.0.0.230:7043
Press CTRL+C to quit
127.0.0.1 - - [12/Jul/2024 10:42:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2024 10:42:04] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Jul/2024 10:42:30] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jul/2024 10:42:30] "POST /upload HTTP/1.1" 200 -
